# Week 4 Exercise — Python to Go Converter (Emmanuel)

Convert Python code to **Go** using an LLM. Paste Python in the input box, choose a model (OpenAI or Ollama), and click **Convert** to generate equivalent Go code in the output box. You can run the Python locally and, if you have the Go toolchain, compile and run the generated Go code.

**Requirements:** For OpenAI models set `OPENAI_API_KEY`. For Ollama models run `ollama serve` and `ollama pull <model>`.

In [ ]:
import os
import io
import sys
import subprocess
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI() if openai_api_key else None

OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

# Model dropdown: (label, model id). OpenAI first, then Ollama.
MODEL_CHOICES = [
    ("OpenAI — GPT-4.1 Mini", "gpt-4.1-mini"),
    ("OpenAI — GPT-4o", "gpt-4o"),
    ("OpenAI — GPT-4 Turbo", "gpt-4-turbo"),
    ("Ollama — Llama 3.2", "llama3.2"),
    ("Ollama — Llama 3.1", "llama3.1"),
    ("Ollama — Mistral", "mistral"),
    ("Ollama — Qwen 2.5 7B", "qwen2.5:7b"),
    ("Ollama — Phi-3", "phi3"),
]
DEFAULT_MODEL_ID = "gpt-4.1-mini"
OLLAMA_MODELS = {"llama3.2", "llama3.1", "mistral", "qwen2.5:7b", "phi3"}

In [ ]:
SYSTEM_PROMPT = """Your task is to convert Python code into correct, idiomatic Go code.
Respond only with Go code. Do not include markdown code fences or explanations.
The Go code must produce the same output as the Python code when run.
Use standard library only unless a well-known package is clearly needed (e.g. math).
Output must be a complete, runnable program: package main with func main().
Preserve behavior: same prints, same logic, same edge cases. For values that may exceed 32 bits (e.g. 2**32), use uint64 in Go."""

In [ ]:
def port(model_id: str, python_code: str) -> str:
    if not python_code or not python_code.strip():
        return "# Paste Python code above and click Convert."
    use_ollama = model_id in OLLAMA_MODELS
    client = ollama_client if use_ollama else openai_client
    if not client:
        return "Set OPENAI_API_KEY for OpenAI, or run Ollama for open-source models."
    user_msg = f"Convert this Python code to Go. Output only Go code, no markdown or explanation.\n\n```python\n{python_code.strip()}\n```"
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ]
    try:
        response = client.chat.completions.create(
            model=model_id, messages=messages, max_tokens=4096
        )
        reply = (response.choices[0].message.content or "").strip()
        for fence in ("```go", "```Go", "```"):
            reply = reply.replace(fence, "")
        return reply.strip()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
def run_python(code: str) -> str:
    if not code or not code.strip():
        return "(No code to run)"
    buf = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buf
    try:
        exec(code.strip(), {"__builtins__": __builtins__})
        return buf.getvalue() or "(No output)"
    except Exception as e:
        return f"Error: {e}"
    finally:
        sys.stdout = old_stdout


def run_go(go_code: str) -> str:
    if not go_code or not go_code.strip() or go_code.startswith("#") or "Error:" in go_code:
        return "(Generate Go code first, then click Run Go.)"
    workdir = os.getcwd()
    main_go = os.path.join(workdir, "main.go")
    try:
        with open(main_go, "w") as f:
            f.write(go_code)
        subprocess.run(
            ["go", "build", "-o", "main", "main.go"],
            check=True, cwd=workdir, capture_output=True, text=True, timeout=30
        )
        result = subprocess.run(
            ["./main"] if os.name != "nt" else [".\\main.exe"],
            cwd=workdir, capture_output=True, text=True, timeout=10
        )
        out = (result.stdout or "") + (result.stderr or "")
        return out.strip() or "(No output)"
    except FileNotFoundError:
        return "Go toolchain not found. Install Go or run the generated code in https://go.dev/play/"
    except subprocess.CalledProcessError as e:
        return (e.stderr or e.stdout or str(e)).strip()
    except Exception as e:
        return str(e)

In [ ]:
with gr.Blocks(title="Python to Go", theme=gr.themes.Soft()) as demo:
    gr.Markdown("### Python → Go converter — paste code, pick a model, convert.")
    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=MODEL_CHOICES,
            value=DEFAULT_MODEL_ID,
            label="Model",
            allow_custom_value=False,
        )
    with gr.Row(equal_height=True):
        python_input = gr.Code(
            label="Python (input)",
            value='def greet(name):\n    print(f"Hello, {name}!")\ngreet("World")',
            language="python",
            lines=20,
        )
        go_output = gr.Code(
            label="Go (output)",
            value="",
            language="javascript",
            lines=20,
        )
    with gr.Row():
        convert_btn = gr.Button("Convert to Go", variant="primary")
        run_py_btn = gr.Button("Run Python")
        run_go_btn = gr.Button("Run Go")
    with gr.Row():
        python_result = gr.TextArea(label="Python run result", lines=6, interactive=False)
        go_result = gr.TextArea(label="Go run result", lines=6, interactive=False)

    convert_btn.click(fn=port, inputs=[model_dropdown, python_input], outputs=[go_output])
    run_py_btn.click(fn=run_python, inputs=[python_input], outputs=[python_result])
    run_go_btn.click(fn=run_go, inputs=[go_output], outputs=[go_result])

demo.launch()